# CNN for MNIST

Das Basis CNN - Projekt auf https://keras.io/examples/mnist_cnn/

## Imports

In [0]:
#from __future__ import print_function # wird bei python 3 nicht benötigt
import keras
from keras.datasets import mnist # unser Datensatz
from keras.models import Sequential # damit wir überhaupt ein Model bauen können
from keras.layers import Dense, Dropout, Flatten # für das "standard" Network
from keras.layers import Conv2D, MaxPooling2D # für das Convolutional Network
from keras import backend as K # für low-level operations (vgl. https://keras.io/backend/)
from keras.optimizers import Adadelta # wie optimiert werden soll

Using TensorFlow backend.


## Parameter definition
Die einzelnen Parameter als globals definiert
* *batch* die Korrekturen die durchgeührt werden (je niedriger, desto mehr)
* *num_classes* Wieviele Kategorien am Ende über bleiben sollen
* *epochs* ist die Anzahl der Durchläufe
* *input image dimensions* wie groß die einzelnen Bilder sind

In [0]:
batch_size = 128
num_classes = 10 
epochs = 1 # hier vorerst auf 1 gesetzt, damit man besser andere Parameter verändern kann

# input image dimensions
img_rows, img_cols = 28, 28

## Train/Test Split

* Zuerst der Split und gleichzeitig das Laden der Daten

* Danach wird das Datenformat überprüft, denn der Input muss dementsprechend angepasst werden
 * vgl. https://keras.io/layers/convolutional/

    data_format: A string, one of "channels_last" or "channels_first". The ordering of the dimensions in the inputs. "channels_last" corresponds to inputs with shape (batch, height, width, channels) while "channels_first" corresponds to inputs with shape (batch, channels, height, width). It defaults to the image_data_format value found in your Keras config file at ~/.keras/keras.json. If you never set it, then it will be "channels_last".

* Train und Test wird in float umgewandelt und durch 255 dividiert und das Ergebnis zugewiesen
* Ausgabe wie unsere Train/Test Daten ausschauen
 * NB! x_train shape: (60000, 28, 28, 1) deutet auf das Datenformat "channels_last"

* Anschließend muss noch der Output-vector in eine binary class matrix convertiert werden
 * vgl. https://keras.rstudio.com/reference/to_categorical.html

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Work on the Model

### Create Model

* Sequential ist das leere Gerüst (The Sequential model is a linear stack of layers)
* mit .add() werden Layer hinzugefügt - dabei wird angegeben von welchem Typ der Layer ist
  * z.B. Conv2D, oder Dense
  * Übergeben werden:
    * Anzahl der Nodes - selbsterklärend
    * kernel_size - kernel sind die kleinen "Stichproben" die das Bild Stück für Stück durchgehen, hier wird deren Größe angegeben (Könnte man eventuell besser formulieren)
    * activation Methode - 
    * im ersten Layer muss auch der input_shape übergeben werden
* in unserem Fall zwei Conv. Layer (32 und 64 Nodes)
* dann folgt MaxPooling - beim Max Pooling wird jeweils ein kleines Feld (2x2 bei uns) mit den max Werten gefüllt
  * also eine Dimensionsreduktion
* mit Dropout werden Nodes gestrichen um ein Overfitting zu verhindern
  * Die Rate wird mit angegeben (0.25)
* Flatten bedeutet, dass aus der Feature Matrix, die aus dem Pooling kommt ein Feature Vector wird
  * wird vor einem fully connected Layer (dense) gemacht
* Es folgt der Dense Layer - übernimmt die Ergebnisse aus dem Convolutional Layers um eine Prediction zu erzeugen
  * bei uns sind es zwei dense Layer mit einem Dropout Zwischenschritt (0.5) bevor die wirkliche Prediction gemacht wird.
    * ester dense Layer mit 128 Nodes und einer relu
    * der Zweite hat (natürlich) nur noch so viele Nodes wie Nummernklassen und eine softmax activation
      * softmax brauch ich zum klassifizieren - da sigmoid ja nur zwei Klassen hätte. Softmax bringt mir den Output auf zwischen 0 und 1 *und* schaut auch, dass die Summe der Outputs eins ist.

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

### Trainingsvorbereitung

Nachdem das Model erstellt ist wird es hier auf das Training vorbereitet - ihm wird gesagt wie es lernen soll.
Dazu brauchen wir:
* Eine loss-function - in unserem Fall: categorical_crossentropy
* Einen Optimizer - Adadelta
* Eine Metrik nach der optimiert werden soll

Es gibt eine Unzahl an Möglichkeiten hier Sachen zu verändern - und ehrlich gesagt fühle ich mich bei weitem noch nicht in der Lage hier groß mitzureden. Deshalb sollten die voreingestellten belassen werden - Adadelta und categorical_crossentropy scheinen fü dieses Problem auch die gängigsten zu sein

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

### Das eigentliche Training

Abhängig von den Parametern kann das sehr viel Zeit in Anspruch nehmen

Die fit Methode ist zum größten Teil selbsterklärend
* verbose ist die Visualisierung des Fortschrittes (um sicher zu gehen, dass der Computer noch rechnet)
* (0 = silent (man sieht nichts), 1 = animated progress bar ([==========]), 2 = zeigt die Epochennummer an (Epoch 1/10))

Dann wird mit evaluate loss und metrics value (bei und accuracy) überprüft und ausgegeben

In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 194s 3ms/step - loss: 0.2670 - accuracy: 0.9187 - val_loss: 0.0627 - val_accuracy: 0.9805
Test loss: 0.06270357351452112
Test accuracy: 0.9804999828338623
